In [1]:
!pip uninstall -y ipywidgets azureml-widgets
!pip install ipywidgets==7.6.5 azureml-widgets --no-cache-dir

Found existing installation: ipywidgets 7.0.0
Uninstalling ipywidgets-7.0.0:
  Successfully uninstalled ipywidgets-7.0.0
Found existing installation: azureml-widgets 1.60.0
Uninstalling azureml-widgets-1.60.0:
  Successfully uninstalled azureml-widgets-1.60.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.8/121.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 130.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 152.8 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.0.8
    Uninstalling widgetsnbextension-3.0.8:
      Successfully uninstalled widgetsnbextension-3.0.8


In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-283693
Azure region: southcentralus
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-283693


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "my-project2-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

vm_size = "STANDARD_D2_V2"
max_nodes = 4
idle_seconds_before_scaledown = 120

try:
    aml_compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f"Found existing compute target: {cluster_name}")
except ComputeTargetException:
    print(f"Compute target '{cluster_name}' not found. Creating a new one...")

    compute_config = AmlCompute.provisioning_configuration(
        vm_size=vm_size,
        max_nodes=max_nodes,
        min_nodes=0,
        idle_seconds_before_scaledown=idle_seconds_before_scaledown
    )
    
    aml_compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    print(f"Submitting compute creation request for '{cluster_name}'...")

    aml_compute_target.wait_for_completion(show_output=True, min_node_count=0, timeout_in_minutes=2)
    print(f"Compute cluster '{cluster_name}' provisioned successfully.")

print("\nCurrent status of the compute cluster:")
print(aml_compute_target.get_status().serialize())

Compute target 'my-project2-cluster' not found. Creating a new one...
Submitting compute creation request for 'my-project2-cluster'...
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Compute cluster 'my-project2-cluster' provisioned successfully.

Current status of the compute cluster:
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2025-06-17T21:54:28.421000+00:00', 'errors': None, 'creationTime': '2025-06-17T21:54:19.324384+00:00', 'modifiedTime': '2025-06-17T21:54:29.985612+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeSc

In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
        '--C': uniform(0.01, 100),
        '--max_iter': choice(50, 100, 200, 300)
    }
)

# Specify a Policy
### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=3, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
### YOUR CODE HERE ###
src = ScriptRunConfig(
    source_directory='./',
    script='train.py',
    environment=sklearn_env,
    compute_target='my-project2-cluster'
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_concurrent_runs=4,
    max_total_runs=20
)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

Matplotlib is building the font cache; this may take a moment.
2025-06-17 21:55:56.758455: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750197358.486940    4122 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750197359.111295    4122 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750197364.193831    4122 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750197364.193882    4122 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750197364.1

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_5918806f-ecc1-4d3f-a215-b6ec45cc360b
Web View: https://ml.azure.com/runs/HD_5918806f-ecc1-4d3f-a215-b6ec45cc360b?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-283693/workspaces/quick-starts-ws-283693&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2025-06-17T21:54:38.5165855Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2025-06-17T21:54:39.1060841Z][SCHEDULER][INFO]Scheduling job, id='HD_5918806f-ecc1-4d3f-a215-b6ec45cc360b_2' 
[2025-06-17T21:54:39.0026351Z][SCHEDULER][INFO]Scheduling job, id='HD_5918806f-ecc1-4d3f-a215-b6ec45cc360b_0' 
[2025-06-17T21:54:39.1079997Z][SCHEDULER][INFO]Scheduling job, id='HD_5918806f-ecc1-4d3f-a215-b6ec45cc360b_1' 
[2025-06-17T21:54:39.1071426Z][SCHEDULER][INFO]Scheduling job, id='HD_5918806f-ecc1-4d3f-a215-b6ec45cc360b_3' 
[2025-06-17T21:54:39.8955063Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_5918806f-ecc1-4d3f-a215-b6ec45cc360b_1' 
[2025-06-17T21:5

{'runId': 'HD_5918806f-ecc1-4d3f-a215-b6ec45cc360b',
 'target': 'my-project2-cluster',
 'status': 'Completed',
 'startTimeUtc': '2025-06-17T21:54:37.123128Z',
 'endTimeUtc': '2025-06-17T22:16:48.230172Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '4da2e7ae-4797-4b23-9992-5dc0543c7cb9',
  'user_agent': 'python/3.10.11 (Linux-6.8.0-1026-azure-x86_64-with-glibc2.35) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.60.0',
  'best_child_run_id': 'HD_5918806f-ecc1-4d3f-a215-b6ec45cc360b_14',
  'score': '0.9088012139605464',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_5918806f-ecc1-4d3f-a215-b6ec45cc360b_14'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azur

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n Parameters:',parameter_values)

model_file_name = 'hyperdriveBestModel.pkl'
remote_model_path = f'outputs/model.pkl'
local_model_path = os.path.join('./training', model_file_name)

print(f"Downloading model from run {best_run.id} to {local_model_path}...")
best_run.download_file(name=remote_model_path, output_file_path=local_model_path)
print("Model downloaded successfully.")

from azureml.core.model import Model

print(f"Registering model '{model_file_name}' from run {best_run.id}...")

model = Model.register(workspace=ws,
                       model_path=local_model_path, # Path to the downloaded model file
                       model_name="best_hyperdrive_model", # Name for your registered model
                       tags={'run_id': best_run.id, 'accuracy': best_run.get_metrics().get('Accuracy', 'N/A')},
                       description="Logistic Regression model tuned with HyperDrive")

print(f"Model registered successfully with name: {model.name}, version: {model.version}")

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###